In [1]:
from IPython.core.interactiveshell import InteractiveShell  #---- Output all jupyter lab inputs instead of the last one
from IPython.display import Markdown, display
InteractiveShell.ast_node_interactivity = "all"             
import glob #---------------------------------------------------- To read the files or folders in a system directory
from netCDF4 import Dataset #------------------------------------ To read nc , nc4 and hdf4 files
import numpy as np
import datetime
import h5py #---------------------------------------------------- To read hdf5 files
from scipy import spatial #-------------------------------------- To extract the values and indices of k nearest neighbors
import pandas as pd
from ast import literal_eval #----------------------------------- For literal evaluation of a string to extract python objects
from pyproj import Proj, transform #----------------------------- To interconvert different projections
import warnings #------------------------------------------------ To suppress warnings
from photutils.utils import ShepardIDWInterpolator as idw #------ To use Shepard's Inverse Distance Weighing Interpolation tool
import re #------------------------------------------------------ To replace characters in a string
import time as tttt
import cartopy.crs as ccrs
import xarray as xr
import xesmf as xe
import pyresample
import matplotlib.pyplot as plt
warnings.simplefilter('ignore')
def printmd(string):
    display(Markdown(string))

/home/kaushal/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/kaushal/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
in_proj  = Proj('+proj=sinu +R=6371007.181 +nadgrids=@null +wktext') #------ Specify input projection
out_proj = Proj(init='epsg:4326') #----------------------------------------- Specify output projection

def Times(x): #------------------------------------------Extract Time from sounding ID. NOTE the time format is HH:MM:SSSS
    y   = str(x)
    yy  = y[8:]
    yyy = '{}:{}:{}'.format(yy[:2], yy[2:4], yy[4:])
    return yyy

hours = [0, 3000000, 6000000, 9000000, 12000000, 15000000, 18000000, 21000000, 23595900]
def f(x): #---------------------------------------------Extract the hour interval of sif time (which  is in seconds)
    for i in range(len(hours)):
        if (x>hours[i]) and (x<hours[i+1]):
            lb = hours[i]
            ub = hours[i+1]
            return lb,ub
            break

def format_time(t): #----------------------------------- Format the time into HH:MM:SSSS for the raw format HH:MM:SSSSSSSS
    s = t
    return s[:-4]

def nn(latitude_list,longitude_list,target): #---------- Find the index of nearest neighbor (NOTE: absolute difference)
    target_lat, target_lon = target[1], target[0]
    d = [abs(latitude-target_lat) + abs(longitude-target_lon) for latitude,longitude in zip(latitude_list,longitude_list)]
    return np.argmin(d)

data = np.genfromtxt('sn_bound_10deg.txt', skip_header = 7, skip_footer = 3)
def tile_finder(Lat,Lon): #----------------------------- Find modis tile numbers in which the argument lat,lon lies
    in_tile = False
    i = 0
    while(not in_tile):
        in_tile = Lat >= data[i, 4] and Lat <= data[i, 5] and Lon >= data[i, 2] and Lon <= data[i, 3]
        i += 1
    V = str(int(data[i-1, 0])).zfill(2)
    H = str(int(data[i-1, 1])).zfill(2)
    return H,V

def extract_pixel_coordinates(ULx,Uly,LRx,LRy,shape):
    x        = np.linspace(ULx, LRx, shape[0], endpoint=False) + abs((ULx-LRx)/(2*shape[0]))
    y        = np.linspace(ULy, LRy, shape[0], endpoint=False) - abs((ULy-LRy)/(2*shape[0]))
    xx, yy   = np.meshgrid(x,y)
    plon, plat = transform(in_proj, out_proj, xx, yy)
    #mlon, mlat = plon[0], np.array([i[0] for i in plat])
    plon       = plon.flatten()
    plat       = plat.flatten()
    return plon, plat#, mlon, mlat

def temporal_interpolation(time1,val1,time2,val2,timeX):
    df    = pd.DataFrame( [(time1, val1) , (time2, val2)] , columns=['Times','Values'] ) 
    df    = df.set_index('Times')
    df    = pd.Series(df['Values'], index=df.index)
    df.index = pd.to_datetime(df.index)
    inter = df.resample('S').interpolate(method='linear')
    valX  = inter.loc[timeX]
    return valX

sif_file_list     = sorted(glob.glob('OCO2_sif/*.nc4'))    #--------------------------------------------- List of all OCO2 files
calipso_file_list = sorted(glob.glob('OCO2_calipso/*.h5')) #--------------------------------------------- List of all OCO2-CALIPSO files
fpar_file_list    = sorted(glob.glob('MCD15A3H/*.hdf'))    
par_folder_list   = sorted(glob.glob('MCD18A2/*'))
ref_folder_list   = sorted(glob.glob('MCD43A4/*'))
data              = np.genfromtxt('sn_bound_10deg.txt', skip_header = 7, skip_footer = 3) #------ File having tile numbers and IDs

In [3]:
for sif_file in sif_file_list: #---------------------------------------------------------------------------Read one sif file at a time
    sif_date        = datetime.datetime.strptime(sif_file.split('_')[3], '%y%m%d').strftime("%Y-%m-%d") #-----Extract sif date
    sif_julian_day  = datetime.datetime.strptime(sif_file.split('_')[3], '%y%m%d').strftime("%j") #-----------Extract sif julian day
    sif             = Dataset(sif_file, mode='r') #-----------------------------------------------------------Open sif file
    calipso_df_list = [] #------------------------------------------------------------------------------------Create an empty list
    for calipso_file in calipso_file_list: #------------------------------------------------------------------Loop through all calipso files
        calipso_date      = datetime.datetime.strptime(calipso_file.split('_')[5], '%y%m%d').strftime("%Y-%m-%d")
        if calipso_date  == sif_date: #----------------------------------------------------------------------------If calipso date matches sif date,
            calipso       = h5py.File(calipso_file, mode='r') #----------------------------------------------------open the calipso file
            calipso_ID    = calipso['OCO2_sounding_id'                                           ][:]
            calipso_dist  = calipso['matchup_distance_km'                                        ][:]
            calipso_index = calipso['matchup_Xindex'                                             ][:]
            calipso_dfs   = pd.DataFrame({'sounding_id':calipso_ID.flatten(),'Xindex':calipso_index.flatten(),'Xdistance':calipso_dist.flatten()}) #-----Create dataframe with variables
            calipso_dfs[calipso_dfs.Xindex==-999.0] = np.nan #------- Replace missing values with nan
            calipso_dfs.dropna(inplace=True) #----------------------- Drop missing values of Xindex
            calipso_dfs[calipso_dfs.Xdistance>=2.0] = np.nan
            calipso_dfs.dropna(inplace=True)
            calipso_df_list.append(calipso_dfs) #-------------------- Add all calipso dataframes into a list
    calipso_df                     = pd.concat(calipso_df_list, ignore_index = True).drop_duplicates() #---------Create a final calipso dataframe for a day

    cloud_albedo                   = sif.groups['Cloud'].variables['albedo'                 ][:].flatten() #--------Read sif variables and flatten them
    cloud_flag                     = sif.groups['Cloud'].variables['cloud_flag'             ][:].flatten()
    cloud_co2_ratio                = sif.groups['Cloud'].variables['co2_ratio'              ][:].flatten()
    cloud_delta_surface_pressure   = sif.groups['Cloud'].variables['delta_surface_pressure' ][:].flatten()
    cloud_o2_ratio                 = sif.groups['Cloud'].variables['o2_ratio'               ][:].flatten()
    vapor_pressure_deficit         = sif.groups['Meteo'].variables['vapor_pressure_deficit' ][:].flatten()
    temperature_2m                 = sif.groups['Meteo'].variables['2m_temperature'         ][:].flatten()
    temperature_skin               = sif.groups['Meteo'].variables['skin_temperature'       ][:].flatten()
    specific_humidity              = sif.groups['Meteo'].variables['specific_humidity'      ][:].flatten()
    surface_pressure               = sif.groups['Meteo'].variables['surface_pressure'       ][:].flatten()
    wind_speed                     = sif.groups['Meteo'].variables['wind_speed'             ][:].flatten()
    continuum_radiance_757nm       = sif.variables         ['continuum_radiance_757nm'      ][:].flatten()
    continuum_radiance_771nm       = sif.variables         ['continuum_radiance_771nm'      ][:].flatten()
    daily_correction_factor        = sif.variables         ['daily_correction_factor'       ][:].flatten()
    footprint                      = sif.variables         ['footprint'                     ][:].flatten()
    IGBP_index                     = sif.variables         ['IGBP_index'                    ][:].flatten()
    latitude                       = sif.variables         ['latitude'                      ][:].flatten()
    longitude                      = sif.variables         ['longitude'                     ][:].flatten()
    measurement_mode               = sif.variables         ['measurement_mode'              ][:].flatten()
    orbit_number                   = sif.variables         ['orbit_number'                  ][:].flatten()
    reduced_chi2_757nm             = sif.variables         ['reduced_chi2_757nm'            ][:].flatten()
    reduced_chi2_771nm             = sif.variables         ['reduced_chi2_771nm'            ][:].flatten()
    sensor_azimuth_angle           = sif.variables         ['sensor_azimuth_angle'          ][:].flatten()
    sensor_zenith_angle            = sif.variables         ['sensor_zenith_angle'           ][:].flatten()
    SIF_757nm                      = sif.variables         ['SIF_757nm'                     ][:].flatten() #----------Use this condition for next block
    SIF_757nm_relative             = sif.variables         ['SIF_757nm_relative'            ][:].flatten()
    SIF_757nm_uncert               = sif.variables         ['SIF_757nm_uncert'              ][:].flatten()
    SIF_771nm                      = sif.variables         ['SIF_771nm'                     ][:].flatten()
    SIF_771nm_relative             = sif.variables         ['SIF_771nm_relative'            ][:].flatten()
    SIF_771nm_uncert               = sif.variables         ['SIF_771nm_uncert'              ][:].flatten()
    solar_azimuth_angle            = sif.variables         ['solar_azimuth_angle'           ][:].flatten()
    solar_zenith_angle             = sif.variables         ['solar_zenith_angle'            ][:].flatten()
    sounding_id                    = sif.variables         ['sounding_id'                   ][:].flatten()
    surface_altitude               = sif.variables         ['surface_altitude'              ][:].flatten()
    time                           = sif.variables         ['time'                          ][:].flatten()
    uncorrected_SIF_757nm          = sif.variables         ['uncorrected_SIF_757nm'         ][:].flatten()
    uncorrected_SIF_757nm_relative = sif.variables         ['uncorrected_SIF_757nm_relative'][:].flatten()
    uncorrected_SIF_771nm          = sif.variables         ['uncorrected_SIF_771nm'         ][:].flatten()
    uncorrected_SIF_771nm_relative = sif.variables         ['uncorrected_SIF_771nm_relative'][:].flatten()
      
    sif_rows  = [(SIF_757nm[i], cloud_albedo[i], cloud_flag[i], cloud_co2_ratio[i], cloud_delta_surface_pressure[i], cloud_o2_ratio[i], vapor_pressure_deficit[i],
                  temperature_2m[i], temperature_skin[i], specific_humidity[i], surface_pressure[i], wind_speed[i], continuum_radiance_757nm[i],
                  continuum_radiance_771nm[i],daily_correction_factor[i], footprint[i], IGBP_index[i], latitude[i], longitude[i], measurement_mode[i],
                  orbit_number[i], reduced_chi2_757nm[i],reduced_chi2_771nm[i], sensor_azimuth_angle[i], sensor_zenith_angle[i], SIF_757nm_relative[i],
                  SIF_757nm_uncert[i], SIF_771nm[i],SIF_771nm_relative[i], SIF_771nm_uncert[i], solar_azimuth_angle[i], solar_zenith_angle[i], sounding_id[i],
                  surface_altitude[i], time[i], uncorrected_SIF_757nm[i], uncorrected_SIF_757nm_relative[i], uncorrected_SIF_771nm[i], uncorrected_SIF_771nm_relative[i])
                  for i in range(0,len(sounding_id))]
    
    column_labels = ['SIF_757nm', 'cloud_albedo', 'cloud_flag', 'cloud_co2_ratio', 'cloud_delta_surface_pressure', 'cloud_o2_ratio', 'vapor_pressure_deficit',
                     'temperature_2m', 'temperature_skin', 'specific_humidity', 'surface_pressure', 'wind_speed', 'continuum_radiance_757nm',
                     'continuum_radiance_771nm','daily_correction_factor', 'footprint', 'IGBP_index', 'latitude', 'longitude', 'measurement_mode',
                     'orbit_number', 'reduced_chi2_757nm','reduced_chi2_771nm', 'sensor_azimuth_angle', 'sensor_zenith_angle',
                     'SIF_757nm_relative', 'SIF_757nm_uncert', 'SIF_771nm','SIF_771nm_relative', 'SIF_771nm_uncert', 'solar_azimuth_angle', 'solar_zenith_angle',
                     'sounding_id', 'surface_altitude', 'time','uncorrected_SIF_757nm', 'uncorrected_SIF_757nm_relative', 'uncorrected_SIF_771nm',
                     'uncorrected_SIF_771nm_relative']
     
    sif_df                         = pd.DataFrame(sif_rows,columns = column_labels) #-------- Create sif variables' dataframe
    calipso_sif_merger             = pd.merge(sif_df, calipso_df, on = ['sounding_id'], how = 'inner') #--------Merge sif and calipso on sounding _id
    calipso_sif_merger['Date']     = calipso_sif_merger['sounding_id'].map(lambda x: '-'.join([str(x)[:4],str(x)[4:6],str(x)[6:]])[:10]) #-----Create new date column
    calipso_sif_merger['SIF_Time'] = calipso_sif_merger['sounding_id'].map(lambda x: Times(x))  #----------------------------------------------Create new time column
    calipso_sif_merger['tile_h'  ] = calipso_sif_merger.apply(lambda x: tile_finder(x['latitude'], x['longitude'])[0], axis=1) #------Create new horizontal tile column
    calipso_sif_merger['tile_v'  ] = calipso_sif_merger.apply(lambda x: tile_finder(x['latitude'], x['longitude'])[1], axis=1) #------Create new vertical tile column
    calipso_sif_merger             = calipso_sif_merger.dropna(how='any')

    grp         = calipso_sif_merger.groupby(['tile_h', 'tile_v']).agg(lambda x: list(x))  #----Group sif-calipso merger(from now on called SIF*) by tile id
    grp         = grp.reset_index() #-----------------------------------------------------------Reset indices
    l_ungrouped = len(calipso_sif_merger)
    l_grouped   = len(grp)
    df          = grp.copy() #--------make a copy of the grouped file
    print('------------------------------------------------------------------------------------------------------------------------------------------------')
    printmd('**For {}, there are {} sif footprints scattered over {} tiles.**'.format(sif_date, l_ungrouped, l_grouped))
    print('------------------------------------------------------------------------------------------------------------------------------------------------')  


    shapes              = [(2400,2400)]#, (1200,1200), (800,800), (600,600), (400,400), (200,200)]
    for shape2 in shapes:
        print('Processing shape : ', shape2[0],'x',shape2[1])
        a                   = tttt.time()
        print('REF Processing Started...\n')
        Each_Ref_Tile_Data  = []
        for index,h_sif,v_sif,sif_lon,sif_lat,sif_time,sif_sid in zip(df.index,df['tile_h'],df['tile_v'],df['longitude'],df['latitude'],df['SIF_Time'],df['sounding_id']):
            print('h{}v{}'.format(h_sif,v_sif), flush = True, sep=',', end=' ')

            for folder_number in range(len(ref_folder_list)):
                ref_julian_day    = ref_folder_list[folder_number].split('/')[1]

                if sif_julian_day == ref_julian_day:
                    ref_file_list = glob.glob(ref_folder_list[folder_number]+'/*.hdf')

                    for num3,ref_file in enumerate(ref_file_list):
                        h_ref = ref_file.split('.')[2][1:3]
                        v_ref = ref_file.split('.')[2][4:6]

                        if (h_ref==h_sif) and (v_ref==v_sif):
                            ref              = Dataset(ref_file, mode='r')
                            nrb1             = ref.variables['Nadir_Reflectance_Band1'][:]
                            fv1              = ref.variables['Nadir_Reflectance_Band1']._FillValue
                            nrb2             = ref.variables['Nadir_Reflectance_Band2'][:]
                            fv2              = ref.variables['Nadir_Reflectance_Band2']._FillValue
                            struct           = getattr(ref, 'StructMetadata.0')
                            struct1          = struct[struct.find('UpperLeftPointMtrs'): struct.find('LowerRightMtrs')][19:-3]
                            struct2          = struct[struct.find('LowerRightMtrs')    : struct.find('Projection')    ][15:-3]
                            ULx, ULy         = literal_eval(struct1)
                            LRx, LRy         = literal_eval(struct2)
                            
                            ref_lon,ref_lat,ref_mlon,ref_mlat = extract_pixel_coordinates(ULx,ULy,LRx,LRy,(2400,2400))
                            
                            
                            t_lat                    = np.linspace(ref_mlat[0], ref_mlat[-1], shape2[0], endpoint=True)
                            t_lon                    = np.linspace(ref_mlon[0], ref_mlon[-1], shape2[1], endpoint=True)                         

                            source_lons, source_lats = np.meshgrid(ref_mlon, ref_mlat)
                            dest_lons, dest_lats     = np.meshgrid(t_lon, t_lat)
                            
                            orig_def                 = pyresample.geometry.SwathDefinition(lons=source_lons, lats=source_lats)
                            targ_def                 = pyresample.geometry.SwathDefinition(lons=dest_lons  , lats=dest_lats  )
                            
                            nrb01                    = nrb1.flatten()
                            wf                       = lambda r: 1/r**2
                            
                            if shape2 != (2400,2400):
                                interp_nrb1    = pyresample.kd_tree.resample_custom(orig_def, nrb1, targ_def, radius_of_influence=500000, neighbours=4, weight_funcs=wf, fill_value=float(fv1))
                                #interp_nrb2    = pyresample.kd_tree.resample_custom(orig_def, nrb2, targ_def, radius_of_influence=500000, neighbours=4, weight_funcs=wf, fill_value=float(fv2))
                                tree           = spatial.KDTree( list(  zip(t_lon.flatten(), t_lat.flatten()) ))
                                for sub in range(len(sif_time)):
                                    target     = (sif_lat[sub] , sif_lon[sub])
                                    ind        = tree.query([(sif_lon[sub],sif_lat[sub])], k=1)[1][0]
                                    Each_Ref_Tile_Data.append((sif_sid[sub], sif_lat[sub],sif_lon[sub],nrb01[ind]))
                            else:
                                tree           = spatial.KDTree( list(  zip(ref_lon.flatten(), ref_lat.flatten()) ))
                                for sub in range(len(sif_time)):
                                    target     = (sif_lat[sub] , sif_lon[sub])
                                    ind        = tree.query([(sif_lon[sub],sif_lat[sub])], k=1)[1][0]
                                    Each_Ref_Tile_Data.append((sif_sid[sub], sif_lat[sub],sif_lon[sub],nrb01[ind]))


        DG = pd.DataFrame(np.array(Each_Ref_Tile_Data),columns=['sounding_id','latitude','longitude','nrb1'])
        DG.to_csv('Optimization/{}_ref_{}.csv'.format(sif_date,shape2[0]),index=False)
        print('\n')
        b  = tttt.time()
        printmd('**{} minutes**'.format((b-a)/60))

------------------------------------------------------------------------------------------------------------------------------------------------


**For 2018-05-01, there are 12081 sif footprints scattered over 46 tiles.**

------------------------------------------------------------------------------------------------------------------------------------------------
Processing shape :  2400 x 2400
REF Processing Started...

h10v05 h10v06 h10v08 h10v09 h10v10 h11v03 h11v04 h11v11 h12v02 h12v03 h12v12 h12v13 h13v01 h13v02 h13v13 h14v01 h17v00 h18v00 h18v01 h18v03 h18v04 h19v01 h19v02 h19v06 h19v07 h20v03 h20v08 h20v09 h20v10 h20v11 h20v12 h21v02 h21v04 h22v04 h23v03 h23v06 h24v04 h25v04 h27v12 h28v06 h28v11 h29v07 h29v08 h29v10 h29v11 h30v09 



**14.973532299200693 minutes**

Processing shape :  1200 x 1200
REF Processing Started...

h10v05 h10v06 h10v08 h10v09 h10v10 h11v03 h11v04 h11v11 h12v02 h12v03 h12v12 h12v13 h13v01 h13v02 h13v13 h14v01 h17v00 h18v00 h18v01 h18v03 h18v04 h19v01 h19v02 h19v06 h19v07 h20v03 h20v08 h20v09 h20v10 h20v11 h20v12 h21v02 h21v04 h22v04 h23v03 h23v06 h24v04 h25v04 h27v12 h28v06 h28v11 h29v07 h29v08 h29v10 h29v11 h30v09 



...

**1.6826311031977335 minutes**

Processing shape :  400 x 400
REF Processing Started...

h07v07 h08v05 h08v06 h08v07 h09v04 h10v03 h11v02 h12v02 h13v03 h13v04 h13v09 h13v10 h13v11 h14v01 h14v02 h14v03 h15v01 h16v00 h16v01 h16v02 h17v00 h17v01 h17v04 h17v05 h17v06 h17v07 h17v08 h18v00 h18v01 h18v02 h19v01 h19v02 h19v03 h20v01 h20v02 h20v04 h21v06 h22v02 h22v03 h22v06 h22v07 h22v08 h22v11 h23v04 h24v03 h24v05 h26v04 h26v06 h27v07 h27v08 h27v09 h31v07 h32v09 



**1.5997619350751242 minutes**

Processing shape :  200 x 200
REF Processing Started...

h07v07 h08v05 h08v06 h08v07 h09v04 h10v03 h11v02 h12v02 h13v03 h13v04 h13v09 h13v10 h13v11 h14v01 h14v02 h14v03 h15v01 h16v00 h16v01 h16v02 h17v00 h17v01 h17v04 h17v05 h17v06 h17v07 h17v08 h18v00 h18v01 h18v02 h19v01 h19v02 h19v03 h20v01 h20v02 h20v04 h21v06 h22v02 h22v03 h22v06 h22v07 h22v08 h22v11 h23v04 h24v03 h24v05 h26v04 h26v06 h27v07 h27v08 h27v09 h31v07 h32v09 



**1.5701183438301087 minutes**

In [ ]:

                            
                            if shape2 == (2400,2400):
                                ref_lonx, ref_latx = ref_lon, ref_lat
                                ref_lon , ref_lat  = ref_lonx[(nrb1x.mask==False) & (nrb2x.mask==False)] , ref_latx[(nrb1x.mask==False) & (nrb2x.mask==False)]
                                
                            tree             = spatial.KDTree( list(  zip(ref_lon, ref_lat) ))
                            
                            for sub in range(len(sif_time)):
                                target         = (sif_lat[sub] , sif_lon[sub])
                                neigh10        = tree.query([(sif_lon[sub], sif_lat[sub])], k=4)[1][0] #---Find k=3 nearest spatial neighbors indices at target
                                #neigh10
                                lon_for_idw    = [ref_lon[i] for i in neigh10] #----------------------------Extract longitudes at these indices
                                lat_for_idw    = [ref_lat[i] for i in neigh10] #----------------------------Extract latitudes also
                                coors_for_idw  = [(i,j) for i,j in zip(lat_for_idw,lon_for_idw)] 
                                
                                nrb_01          = [nrb1[i] for i in neigh10]  #---------------------------Find variable values at nearest neighbors
                                nrb_02          = [nrb2[i] for i in neigh10]
                                func_nrb_01     = idw(coors_for_idw, nrb_01) #---------------------------Create inverse distance weighing (IDW) interpolation function at nn coordinates
                                func_nrb_02     = idw(coors_for_idw, nrb_02)
                                interp_nrb1     = func_nrb_01(target)        #-----------------------------------Find interpolated hourly par at target
                                interp_nrb2     = func_nrb_02(target)

                                Each_Ref_Tile_Data.append((sif_sid[sub], sif_lat[sub], sif_lon[sub], interp_nrb1, interp_nrb2))


        DG = pd.DataFrame(np.array(Each_Ref_Tile_Data),columns=['sounding_id','latitude','longitude','nrb1','nrb2'])
        DG.to_csv('Optimization/{}_ref_{}.csv'.format(sif_date,shape2[0]),index=False)
        print('\n')
        b  = tttt.time()
        printmd('**{} minutes**'.format((b-a)/60))

In [23]:
day               = '03'
df                = pd.read_csv('Optimization/2018-05-{}_ref_2400.csv'.format(day))
df['sounding_id'] = df['sounding_id'].map(lambda x: int(x))

list_of_refs = sorted(glob.glob('Optimization/2018-05-{}_*.csv'.format(day)))
list_of_refs.remove('Optimization/2018-05-{}_ref_2400.csv'.format(day))

for i in list_of_refs:
    dg                = pd.read_csv(i)
    dg['sounding_id'] = dg['sounding_id'].map(lambda x: int(x))   
    dg['1_p']         = abs(dg['nrb1'] - df['nrb1']) * 100 / df['nrb1']
    #dg['2_p']         = abs(dg['nrb2'] - df['nrb2']) * 100 / df['nrb2']
    dg                = dg.rename({'1_p':'1_p_{}'.format(i[28:-4])},axis=1)#, '2_p':'2_p_{}'.format(i[28:-4])}, axis=1)
    dg.to_csv('Optimization/2018-05-{}_REF_{}.csv'.format(day,i[28:-4]), index=False)
    del dg

da = pd.read_csv('Optimization/2018-05-{}_REF_200.csv'.format(day) )
db = pd.read_csv('Optimization/2018-05-{}_REF_400.csv'.format(day) )
dc = pd.read_csv('Optimization/2018-05-{}_REF_600.csv'.format(day) )
dd = pd.read_csv('Optimization/2018-05-{}_REF_800.csv'.format(day) )
de = pd.read_csv('Optimization/2018-05-{}_REF_1200.csv'.format(day))
df = pd.read_csv('Optimization/2018-05-{}_ref_2400.csv'.format(day))

dx1 = df. merge(da, on=['sounding_id','latitude','longitude'], how='inner')
dx2 = dx1.merge(db, on=['sounding_id','latitude','longitude'], how='inner')
dx3 = dx2.merge(dc, on=['sounding_id','latitude','longitude'], how='inner')
dx4 = dx3.merge(dd, on=['sounding_id','latitude','longitude'], how='inner')
dx5 = dx4.merge(de, on=['sounding_id','latitude','longitude'], how='inner')

dx5['sounding_id'].nunique()
#dx6 = dx5[['sounding_id','1_p_200','2_p_200','1_p_400','2_p_400','1_p_600','2_p_600','1_p_800','2_p_800','1_p_1200','2_p_1200',]]
dx6 = dx5[['sounding_id','1_p_200','1_p_400','1_p_600','1_p_800','1_p_1200']]
dx6 = dx6.replace([np.inf, -np.inf], np.nan).dropna(how="any")
dx6.shape
dx7 = dx6.mean()
dx7

9077

(3413, 6)

sounding_id    2.018050e+15
1_p_200        1.471861e+02
1_p_400        1.415891e+02
1_p_600        9.027400e+01
1_p_800        9.623598e+01
1_p_1200       1.424587e+02
dtype: float64

In [20]:
xx    = [200,400,600,800,1200]
yy1_1 = [abs(i) for i in list(dx7[1:])]
#yy1_2 = [abs(i) for i in list(dx7[2::2])]

In [22]:
yy2_1 = [abs(i) for i in list(dx7[1:])]
#yy2_2 = [abs(i) for i in list(dx7[2::2])]

In [24]:
yy3_1 = [abs(i) for i in list(dx7[1:])]
#yy3_2 = [abs(i) for i in list(dx7[2::2])]

In [ ]:
yy4_1 = [abs(i) for i in list(dx7[1::2])]
yy4_2 = [abs(i) for i in list(dx7[2::2])]

In [ ]:
yy5_1 = [abs(i) for i in list(dx7[1::2])]
yy5_2 = [abs(i) for i in list(dx7[2::2])]

In [ ]:
yy6_1 = [abs(i) for i in list(dx7[1::2])]
yy6_2 = [abs(i) for i in list(dx7[2::2])]

In [ ]:
yy7_1 = [abs(i) for i in list(dx7[1::2])]
yy7_2 = [abs(i) for i in list(dx7[2::2])]

In [ ]:
yy8_1 = [abs(i) for i in list(dx7[1::2])]
yy8_2 = [abs(i) for i in list(dx7[2::2])]

In [ ]:
yy9_1 = [abs(i) for i in list(dx7[1::2])]
yy9_2 = [abs(i) for i in list(dx7[2::2])]

In [ ]:
yy10_1 = [abs(i) for i in list(dx7[1::2])]
yy10_2 = [abs(i) for i in list(dx7[2::2])]

##### Graph Reflectance Band 1

In [29]:
%matplotlib

dates = ['2018-05-01','2018-05-02','2018-05-03']#,'2018-05-04','2018-05-05','2018-05-06','2018-05-07','2018-05-08','2018-05-09','2018-05-10']
rfls1 = [yy1_1, yy2_1, yy3_1]#, yy4_1, yy5_1, yy6_1, yy7_1, yy8_1, yy9_1, yy10_1]
#rfls2 = [yy1_2, yy2_2, yy3_2]#, yy4_2, yy5_2, yy6_2, yy7_2, yy8_2, yy9_2, yy10_2]

Using matplotlib backend: Qt5Agg


In [30]:
yy1_1, yy2_1

([103.65855989471396,
  108.87315685528334,
  98.95099009045086,
  100.06857794351984,
  89.12454314520423],
 [77.63754088720547,
  81.02570714604362,
  72.32431530958031,
  73.14937387963286,
  74.18994579335524])

In [31]:
xx

[200, 400, 600, 800, 1200]

In [32]:
rfls1[0]

[103.65855989471396,
 108.87315685528334,
 98.95099009045086,
 100.06857794351984,
 89.12454314520423]

In [33]:
#xx = [200,400,600]
for i,j in zip(rfls1, dates):# , [yy1_2, yy2_2, yy3_2]):
    plt.plot(xx,i, label=j)
plt.legend()
plt.xlabel('Subsamples')
plt.ylabel('Percentage Difference')
plt.title('Reflectance_Band_1_10')
plt.show()

Text(0.5, 0, 'Subsamples')

Text(0, 0.5, 'Percentage Difference')

Text(0.5, 1.0, 'Reflectance_Band_1_10')

##### Graph Reflectance Band 2 

In [ ]:
%matplotlib
for i,j in zip(rfls2, dates):
    plt.plot(xx,i, label=j)
plt.legend()
plt.xlabel('Subsamples')
plt.ylabel('Percentage Difference')
plt.title('Reflectance_Band_2_10')
plt.show()

In [ ]:
!cd Optimization/ && rm -r *.csv

In [ ]:
!pwd